In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install gensim
!pip install python-dotenv
!pip install openai
!pip install pinecone
!pip install umap-learn
!pip install hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 131.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 101.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 128.7 MB/s eta 0:00:00


In [3]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('/home/ec2-user/SageMaker', 'FactChecker-V3'))

if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

2025-01-22 15:37:14.339763: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-22 15:37:19.753632: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-22 15:37:19.817813: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 15:37:27.500445: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-22 15:37:39.402901: W tensorflow/com

In [4]:
# DEPRECATED
# chroma_client = chromadb.PersistentClient(path="./../../Clustering/Clustering/Chroma")
# # Count number of collections
# print(chroma_client.count_collections())
# 
# # Get all collection names
# collection_names = chroma_client.list_collections()
# 
# # Loop through each collection and drop it
# for collection_name in collection_names:
#     if collection_name.name != 'climate_claims_embeddings_unchanged':
#         chroma_client.delete_collection(collection_name.name)

In [21]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
for param in params:
    percentage = 0.75
    data_loader = DataLoader(percentage, True, param['random_seed'])
    use_only_card = param['use_only_CARD']
    size_of_dataset = param['size_of_dataset']
    del param['size_of_dataset']
    del param['use_only_CARD']
    
    train_df, test_df = data_loader.create_reddit_dataset()

    # train_df, test_df = data_loader.create_train_test_df(False, False, False, True, size_of_dataset)
    # if use_only_card:
    #     print("using card")
    #     train_df, test_df = data_loader.create_train_test_df(True, True, True, True, size_of_dataset)

    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    # cluster_df = clf.generate_explanations_and_similar_for_each_claim(cluster_df, 'predicted_veracity', 'cluster', 'text')
    test_df['Source'] = 'Prediction from model'
    combined_df = pd.concat([train_df[['Text', 'Source']], test_df[['Text', 'Source']]])
    cluster_df = pd.merge(cluster_df, combined_df, left_on='text', right_on='Text', how='left')
    cluster_df.drop(columns=['Text'], inplace=True)
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
        'size_of_dataset': size_of_dataset,
        'use_only_CARD': use_only_card
    }
    # Prepend the value 'param' to the keys in params
    for key, value in param.items():
        local_key = 'params.' + key
        output[local_key] = value

    results.append(output)

All data sources are being used
1
Number of experiments to run:  1
Fitting
Getting embeddings...
Getting embeddings for batch  0  out of  36
Getting embeddings for batch  0  out of  403
Getting embeddings for batch  50  out of  403
Getting embeddings for batch  100  out of  403
Getting embeddings for batch  150  out of  403
Getting embeddings for batch  200  out of  403
Getting embeddings for batch  250  out of  403
Getting embeddings for batch  300  out of  403
Getting embeddings for batch  350  out of  403
Getting embeddings for batch  400  out of  403
temp_df length: 439
claims_embeddings shape:  (36, 3072)
claims_embeddings predict shape:  (403, 3072)
Running parametric supervised umap...
seed: 23
Transforming time: 0.6328814029693604 seconds
Running hdbscan...
breaking further...
14
14
14
14
14
14
cluster  predict  veracity
-1       False     3           86
                   1           61
         True     -1           27
 2       False     3            5
 3       False     3   

/home/ec2-user/SageMaker/FactChecker-V3/ClusterAndPredict/ClusterAndPredict.py:482: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Filter df to where veracity is value
/home/ec2-user/SageMaker/FactChecker-V3/ClusterAndPredict/ClusterAndPredict.py:483: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_df = cluster_df[cluster_df['veracity'] == value]
/home/ec2-user/SageMaker/FactChecker-V3/ClusterAndPredict/ClusterAndPredict.py:482: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

happened
ERROR DEBUG
1    36
Name: veracity, dtype: int64
3    31
1     5
Name: predicted_veracity, dtype: int64
happened
ERROR DEBUG
1    9
Name: veracity, dtype: int64
1    5
3    4
Name: predicted_veracity, dtype: int64
happened
ERROR DEBUG
1    36
Name: veracity, dtype: int64
3    31
1     5
Name: predicted_veracity, dtype: int64
happened
ERROR DEBUG
1    9
Name: veracity, dtype: int64
1    5
3    4
Name: predicted_veracity, dtype: int64


/home/ec2-user/SageMaker/FactChecker-V3/ClusterAndPredict/ClusterAndPredict.py:586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/SageMaker/FactChecker-V3/ClusterAndPredict/ClusterAndPredict.py:482: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

happened
ERROR DEBUG
1    36
Name: veracity, dtype: int64
3    31
1     5
Name: predicted_veracity, dtype: int64
happened
ERROR DEBUG
1    9
Name: veracity, dtype: int64
1    5
3    4
Name: predicted_veracity, dtype: int64
happened
ERROR DEBUG
1    36
Name: veracity, dtype: int64
3    31
1     5
Name: predicted_veracity, dtype: int64
happened
ERROR DEBUG
1    9
Name: veracity, dtype: int64
1    5
3    4
Name: predicted_veracity, dtype: int64


/home/ec2-user/SageMaker/FactChecker-V3/ClusterAndPredict/ClusterAndPredict.py:482: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Filter df to where veracity is value
/home/ec2-user/SageMaker/FactChecker-V3/ClusterAndPredict/ClusterAndPredict.py:483: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_df = cluster_df[cluster_df['veracity'] == value]
/home/ec2-user/SageMaker/FactChecker-V3/ClusterAndPredict/ClusterAndPredict.py:482: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [27]:
viz = Visualizer()
df_with_two_dimens = viz.fit_transform(cluster_dfs[0], 'embeddings')
# df_with_two_dimens = df_with_two_dimens.drop('embeddings', axis=1)
cluster_dfs[0] = df_with_two_dimens
df_with_two_dimens

,text,veracity,predict,predicted_veracity,embeddings,cluster,num_correct_in_cluster,total_in_cluster,cluster_accuracy,Source,reduced_embeddings
0,Its emissions is more than the emissions of 14...,1,True,4,"[-0.16720734536647797, 0.3540584146976471, -0....",-1,147,174,0.844828,Prediction from model,"[1.2286924, 5.118195]"
1,"China banned CFCs in 2010, as per what they an...",1,True,4,"[-0.20001590251922607, 0.4432351291179657, -1....",-1,147,174,0.844828,Prediction from model,"[0.73341674, 4.6134048]"
2,Climate Models](https://www.heritage.org/envir...,1,True,1,"[0.4435594975948334, 0.30042654275894165, -0.2...",8,21,21,1.000000,Prediction from model,"[9.652608, 8.03992]"
3,And then there's this monstrous inconsistency ...,1,True,4,"[-0.10906676948070526, -0.027161184698343277, ...",-1,147,174,0.844828,Prediction from model,"[4.241845, 7.897852]"
4,>[Bungling World Bank bureaucrats ](https://ny...,1,True,1,"[0.1842391937971115, 0.5911382436752319, -0.85...",11,70,70,1.000000,Prediction from model,"[-0.46748203, 5.271638]"
...,...,...,...,...,...,...,...,...,...,...,...
434,Increases in average and extreme temperatures ...,3,False,3,"[0.27104878425598145, 0.5601314306259155, -0.7...",13,107,108,0.990741,EPA,"[12.006522, 3.2026904]"
435,About 80% of the ground in Alaska has permafro...,3,False,3,"[-0.1546349674463272, 0.0839909166097641, -0.5...",-1,147,174,0.844828,EPA,"[3.0316122, 6.38583]"
436,When a tree came in contact with electrical di...,3,False,3,"[0.5470941662788391, 0.7620251774787903, -0.75...",-1,147,174,0.844828,EPA,"[12.635266, 5.6658316]"
437,"They also breathe at a faster rate, increasing...",3,False,3,"[0.21261060237884521, 0.5829666256904602, -0.8...",13,107,108,0.990741,EPA,"[7.4739237, 1.9064482]"


In [29]:
import boto3
import pandas as pd
import os

# Create S3 client
s3 = boto3.client('s3')

bucket_name = 'sagemaker-us-east-1-390403859474'
s3_file_key = 'processed_file.csv'  # File path in S3
local_file_path = 'downloaded_file.csv'  # Local file name to save

# Download file (commented out as per original code)
# s3.download_file(bucket_name, s3_file_key, local_file_path)
# print(f"File {s3_file_key} has been downloaded to {local_file_path}.")
cluster_dfs[0].to_csv(local_file_path)

# Read CSV file
results_df = pd.read_csv(local_file_path)
results_df = clf.clean_columns_for_s3(results_df)
results_df = clf.generate_explanations_and_similar_for_each_claim(results_df, 'predicted_veracity', 'cluster', 'text', True)

# Process file path for the entire CSV
processed_file_path = f'results_{pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv'
results_df.to_csv(processed_file_path, index=False)

# Upload the entire CSV file
upload_file_key = 'processed_file.csv'  # Path to upload to S3
s3.upload_file(processed_file_path, bucket_name, upload_file_key)
print(f"File {processed_file_path} has been uploaded to s3://{bucket_name}/{upload_file_key}.")

# Create a local directory for processed files
os.makedirs('processed_files', exist_ok=True)

# Split DataFrame into 1000-row chunks
chunk_size = 1000
chunks = [results_df.iloc[i:i + chunk_size] for i in range(0, len(results_df), chunk_size)]

# Process and upload each chunk
for index, chunk in enumerate(chunks):
    # Generate local file path for the chunk
    local_chunk_path = f'processed_files/processed_file_{index}.json'
    
    # Save chunk to local file
    chunk.to_json(local_chunk_path)
    
    # Generate S3 key for the chunk
    s3_chunk_key = f'processed_files/processed_file_{index}.json'
    
    # Upload chunk to S3
    s3.upload_file(local_chunk_path, bucket_name, s3_chunk_key)
    print(f"File {local_chunk_path} has been uploaded to s3://{bucket_name}/{s3_chunk_key}.")

print("All chunks and the entire CSV file have been processed and uploaded.")

Generating explanations: 100%|██████████| 439/439 [00:15<00:00, 27.61it/s] 


File results_2025-01-22_16-31-37.csv has been uploaded to s3://sagemaker-us-east-1-390403859474/processed_file.csv.
File processed_files/processed_file_0.json has been uploaded to s3://sagemaker-us-east-1-390403859474/processed_files/processed_file_0.json.
All chunks and the entire CSV file have been processed and uploaded.


In [34]:
!pip install opensearch-py
!pip install requests-aws4auth

In [39]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

host = 'qwrgattdrsmspf43ydw6.us-east-1.aoss.amazonaws.com'
region = 'us-east-1'
service = 'aoss'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

index_name = 'my_vector_index'
index_body = {
    'mappings': {
        'properties': {
            'id': {'type': 'keyword'},
            'text': {'type': 'text'},
            'veracity': {'type': 'text'},
            'predict': {'type': 'boolean'},
            'predicted_veracity': {'type': 'text'},
            'cluster': {'type': 'text'},
            'source': {'type': 'text'},
            'embeddings': {
                'type': 'knn_vector',
                'dimension': 100,
                'method': {
                    'name': 'hnsw',
                    'space_type': 'l2',
                    'engine': 'nmslib'
                }
            },
            'reduced_embeddings': {
                'type': 'knn_vector',
                'dimension': 2,
                'method': {
                    'name': 'hnsw',
                    'space_type': 'l2',
                    'engine': 'nmslib'
                }
            },
        }
    }
}

client.indices.create(index=index_name, body=index_body)

from opensearchpy.helpers import bulk

def generate_actions():
    for _, row in df.iterrows():
        text = str(row['text'])
        filtered_text = ''.join(re.findall(r'[A-Za-z0-9]', text))
        id_to_use = filtered_text[:30]
        yield {
            "_index": index_name,
            "_source": {
                "id": id_to_use,
                "text": row['text'],
                "veracity": row['veracity'],
                "predict": row['predict'],
                "predicted_veracity": row['predicted_veracity'],
                "cluster": row['cluster'],
                "source": row['Source'],
                "reduced_embeddings": row['reduced_embeddings'],
                "embedding": row['embedding'].tolist()  # Convert numpy array to list
            }
        }

success, _ = bulk(client, generate_actions())
print(f"Indexed {success} documents")

AttributeError: 'OpenSearchService' object has no attribute 'indices'

Zapier

In [18]:
best_estimator.clusters_df.head(100).to_csv("test1.csv", index=False)

In [ ]:
import pandas as pd
import os
import boto3
from datetime import datetime
import io
import re
import time

# Get access, secret key, and bucket name from environment variables
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY']
AWS_SECRET_KEY = os.environ['AWS_SECRET_KEY']
BUCKET_NAME = os.environ['BUCKET_NAME']

# Data File
CSV_FILE_PATH = processed_file_path

# Connect to S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY
)

# Create local temp file
TEMP_DIR = 'temp_files'
os.makedirs(TEMP_DIR, exist_ok=True)

# Read CSV file and get column name
data = pd.read_csv(CSV_FILE_PATH)

batch_size = 1250
total_batches = (len(data) + batch_size - 1) // batch_size
upload_count = 0

for i in range(total_batches):
    
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(data))
    df = data.iloc[start_idx:end_idx]
    column_names = df.columns
    
    # Create files contains single claim
    for index, row in df.iterrows():
        text = str(df.loc[index, 'text'])
        filtered_text = ''.join(re.findall(r'[A-Za-z0-9]', text))
        sub_name = filtered_text[:30]
        file_name = f"claim_{sub_name}.csv"
        file_path = os.path.join(TEMP_DIR, file_name)
        
        single_row_df = pd.DataFrame([row], columns=column_names)
        single_row_df.to_csv(file_path, index=False)
        
        s3_key = f"groundtruth/{file_name}"
        
        # Check if the file exists in S3
        try:
            s3_client.head_object(Bucket=BUCKET_NAME, Key=s3_key)
            # File exists, download it
            s3_object = s3_client.get_object(Bucket=BUCKET_NAME, Key=s3_key)
            s3_data = pd.read_csv(io.BytesIO(s3_object['Body'].read()))
            
            old_prediction = single_row_df['predicted_veracity'].iloc[0]
            new_prediction = s3_data['predicted_veracity'].iloc[0]
            
            # Compare the data
            if old_prediction == new_prediction and False:
                print(f"{file_name} is up-to-date. Skipping upload.")
                continue
            else:
                print(f"{file_name} is outdated. Uploading new version.")
        except s3_client.exceptions.ClientError as e:
            if e.response['Error']['Code'] == '404':
                # File does not exist
                print(f"{file_name} does not exist in S3. Uploading new file.")
            else:
                raise
        
        # Upload to S3
        s3_client.upload_file(
            Filename=file_path,
            Bucket=BUCKET_NAME,
            Key=s3_key  # S3 path
        )
        print(f"Uploaded {file_name} to S3.")
        upload_count += 1

        # Rest if upload count equals batch_size
        if upload_count == batch_size:
            print("Rest for 180s...")
            time.sleep(120)
            upload_count = 0

# Delete temp files
for file in os.listdir(TEMP_DIR):
    os.remove(os.path.join(TEMP_DIR, file))
os.rmdir(TEMP_DIR)

print("All files uploaded successfully!")

In [ ]:
import boto3
import pandas as pd
import gzip
import json

# Get S3 and object key from environment variables
bucket = os.environ['BUCKET_S3']
bucket_key = os.environ['BUCKET_S3_KEY']


# 使用 boto3 客户端
s3 = boto3.client('s3')

# 下载文件到本地
s3.download_file(bucket, key, 'data.json.gz')

# 解压缩并逐行读取数据
data = []
with gzip.open('data.json.gz', 'rt') as f:
    for line in f:
        data.append(json.loads(line))

# 如果数据为 DataFrame 格式的，可以转为 pandas DataFrame
df = pd.json_normalize(data)

df.to_csv("GoogleFactCheckData.csv")

# 查看数据
print(df.head())


In [ ]:
import boto3
import pandas as pd
import io
import math
import time

def process_claims(claims):
    sentences = claims.split("\n")
    if len(sentences) > 5:
        return "\n".join(sentences[:5]) + "..."
    return claims

# Initialize S3 client
s3 = boto3.client('s3')

# Define bucket names and folders
source_bucket = 'sagemaker-us-east-1-390403859474'
source_folder = 'processed_files/'
destination_bucket = 'testtestgfc'
destination_folder = 'ground_t/'

# List files in the source folder
response = s3.list_objects_v2(Bucket=source_bucket, Prefix=source_folder)
file_keys = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'] != source_folder]

# Load and process all files
all_data = []
for file_key in file_keys:
    obj = s3.get_object(Bucket=source_bucket, Key=file_key)
    df = pd.read_json(io.BytesIO(obj['Body'].read()))
    
    if 'similar_claims' in df.columns:
        df['similar_claims'] = df['similar_claims'].apply(process_claims)
    
    if 'id' in df.columns:
        df['id'] = df['id'].astype(str).str[:100]  # Substring 'id' column to first 100 characters
    
    all_data.append(df)

# Combine all DataFrames
combined_df = pd.concat(all_data, ignore_index=True)

# Calculate number of chunks
chunk_size = 500
num_chunks = math.ceil(len(combined_df) / chunk_size)

# Process and upload chunks
for i in range(num_chunks):
    if i == 0:
        continue
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(combined_df))
    chunk = combined_df.iloc[start_idx:end_idx]
    
    # Save chunk to CSV buffer
    buffer = io.StringIO()
    chunk.to_csv(buffer, index=False)
    buffer.seek(0)
    
    # Generate chunk file name
    chunk_file_name = f"processed_chunk_{i+1}.csv"
    destination_key = destination_folder + chunk_file_name
    
    # Upload chunk to destination bucket
    s3.put_object(Bucket=destination_bucket, Key=destination_key, Body=buffer.getvalue())
    print(f"Uploaded chunk {i+1}/{num_chunks}: s3://{destination_bucket}/{destination_key}")
    time.sleep(275)

print("All data has been processed and uploaded in chunks.")


Uploaded chunk 2/17: s3://testtestgfc/ground_t/processed_chunk_2.csv
Uploaded chunk 3/17: s3://testtestgfc/ground_t/processed_chunk_3.csv
Uploaded chunk 4/17: s3://testtestgfc/ground_t/processed_chunk_4.csv
Uploaded chunk 5/17: s3://testtestgfc/ground_t/processed_chunk_5.csv
Uploaded chunk 6/17: s3://testtestgfc/ground_t/processed_chunk_6.csv
Uploaded chunk 7/17: s3://testtestgfc/ground_t/processed_chunk_7.csv
Uploaded chunk 8/17: s3://testtestgfc/ground_t/processed_chunk_8.csv
Uploaded chunk 9/17: s3://testtestgfc/ground_t/processed_chunk_9.csv
Uploaded chunk 10/17: s3://testtestgfc/ground_t/processed_chunk_10.csv
Uploaded chunk 11/17: s3://testtestgfc/ground_t/processed_chunk_11.csv
Uploaded chunk 12/17: s3://testtestgfc/ground_t/processed_chunk_12.csv
Uploaded chunk 16/17: s3://testtestgfc/ground_t/processed_chunk_16.csv
Uploaded chunk 17/17: s3://testtestgfc/ground_t/processed_chunk_17.csv
All data has been processed and uploaded in chunks.
